[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/rag/haystack_mongodb_cooking_advisor_pipeline.ipynb)


# Haystack and MongoDB Atlas RAG notebook

Install dependencies:

In [11]:
pip install haystack-ai mongodb-atlas-haystack tiktoken datasets

  Using cached haystack_ai-2.2.1-py3-none-any.whl (345 kB)
  Using cached mongodb_atlas_haystack-0.3.0-py3-none-any.whl (13 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached datasets-2.19.2-py3-none-any.whl (542 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15


## Setup MongoDB Atlas connection and Open AI


* Set the MongoDB connection string. Follow the steps [here](https://www.mongodb.com/docs/manual/reference/connection-string/) to get the connection string from the Atlas UI.

* Set the OpenAI API key. Steps to obtain an API key as [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)

In [3]:
import getpass
import os
import re

In [8]:
conn_str = getpass.getpass("Enter your MongoDB connection string:")
conn_str = conn_str.strip()
conn_str = (
    re.sub(
        r"appName=[^&\s]*",
        "appName=devrel.content.python",
        conn_str,
        flags=re.IGNORECASE,
    )
    if re.search(r"appName=[^&\s]*", conn_str, re.IGNORECASE)
    else conn_str + ("&" if "?" in conn_str else "?") + "appName=devrel.content.python"
)
os.environ["MONGO_CONNECTION_STRING"] = conn_str

Enter your MongoDB connection string:··········


In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your Open AI Key:")

Enter your Open AI Key:··········


## Create vector search index on collection

Follow this [tutorial](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/) to create a vector index on database: `ai_shop` collection `test_collection`.

Verify that the index name is `vector_index` and the syntax specify:
```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
}
```

### Setup vector store to load documents:

In [12]:
from bson import json_util
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.components.retrievers.mongodb_atlas import (
    MongoDBAtlasEmbeddingRetriever,
)
from haystack_integrations.document_stores.mongodb_atlas import (
    MongoDBAtlasDocumentStore,
)

# Create some example documents
# documents = [
#     Document(content="My name is Jean and I live in Paris."),
#     Document(content="My name is Mark and I live in Berlin."),
#     Document(content="My name is Giorgio and I live in Rome."),
# ]

# dataset = load_dataset("MongoDB/product-catalog")

dataset = {
    "train": [
        {
            "title": "Spinach Lasagna Sheets",
            "price": "$3.50",
            "description": "Infused with spinach, these sheets add a pop of color and extra nutrients.",
            "category": "Pasta",
            "emoji": "📗",
        },
        {
            "title": "Gluten-Free Lasagna Sheets",
            "price": "$4.00",
            "description": "Perfect for those with gluten intolerance, made with a blend of rice and corn flour.",
            "category": "Pasta",
            "emoji": "🍚🌽",
        },
        {
            "title": "Mascarpone",
            "price": "$4.00",
            "description": "Creamy and rich, this cheese adds a luxurious touch to lasagna.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Queso Fresco",
            "price": "$3.00",
            "description": "A mild, crumbly cheese that can be a suitable replacement for ricotta.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Vegetarian Lentil Sauce",
            "price": "$4.00",
            "description": "A meatless option made with cooked lentils that mimics the texture of ground meat.",
            "category": "Vegetarian",
            "emoji": "🍲",
        },
        {
            "title": "Turkey Bolognese",
            "price": "$5.00",
            "description": "A leaner alternative to beef, turkey provides a lighter but flavorful taste.",
            "category": "Poultry",
            "emoji": "🦃",
        },
        {
            "title": "Mushroom and Walnut Sauce",
            "price": "$5.50",
            "description": "Combining chopped mushrooms and walnuts for a hearty vegetarian filling.",
            "category": "Vegetarian",
            "emoji": "🍄🥜",
        },
        {
            "title": "Chicken Bolognese",
            "price": "$5.00",
            "description": "Ground chicken offers a different twist on the classic meat sauce.",
            "category": "Poultry",
            "emoji": "🐔",
        },
        {
            "title": "Vegan Soy Meat Sauce",
            "price": "$4.50",
            "description": "Made from soy protein, this vegan meat sauce replicates the texture and flavor of traditional meat.",
            "category": "Vegan",
            "emoji": "🌱",
        },
        {
            "title": "Tomato Basil Sauce",
            "price": "$3.50",
            "description": "A tangy alternative to béchamel, made with fresh tomatoes and basil.",
            "category": "Vegetarian",
            "emoji": "🍅",
        },
        {
            "title": "Pesto Cream Sauce",
            "price": "$5.50",
            "description": "A fusion of creamy béchamel and rich basil pesto for a unique flavor.",
            "category": "Dairy",
            "emoji": "🍝",
        },
        {
            "title": "Alfredo Sauce",
            "price": "$4.50",
            "description": "A rich and creamy white sauce made with parmesan and butter.",
            "category": "Dairy",
            "emoji": "🧈",
        },
        {
            "title": "Coconut Milk Béchamel",
            "price": "$4.00",
            "description": "A dairy-free version of the classic béchamel made with coconut milk.",
            "category": "Vegan",
            "emoji": "🥥",
        },
        {
            "title": "Vegan Cashew Cream Sauce",
            "price": "$5.00",
            "description": "A rich and creamy sauce made from blended cashews as a dairy-free alternative.",
            "category": "Vegan",
            "emoji": "🥜",
        },
        {
            "title": "Kale",
            "price": "$2.00",
            "description": "Another leafy green option, kale offers a chewy texture and rich nutrients.",
            "category": "Leafy Greens",
            "emoji": "🥬",
        },
        {
            "title": "Bell Peppers",
            "price": "$2.50",
            "description": "Sliced bell peppers in various colors add sweetness and crunch.",
            "category": "Vegetables",
            "emoji": "🫑",
        },
        {
            "title": "Artichoke Hearts",
            "price": "$3.50",
            "description": "Tender and flavorful, artichoke hearts bring a Mediterranean twist to the dish.",
            "category": "Vegetables",
            "emoji": "🍽️",
        },
        {
            "title": "Spinach",
            "price": "$2.00",
            "description": "Fresh or frozen spinach adds a pop of color and nutrients.",
            "category": "Leafy Greens",
            "emoji": "🥬",
        },
        {
            "title": "Broccoli",
            "price": "$2.50",
            "description": "Small broccoli florets provide texture and a distinct flavor.",
            "category": "Vegetables",
            "emoji": "🥦",
        },
        {
            "title": "Whole Wheat Lasagna Sheets",
            "price": "$3.00",
            "description": "Made from whole wheat grains, these sheets are healthier and provide a nutty flavor.",
            "category": "Pasta",
            "emoji": "🌾",
        },
        {
            "title": "Zucchini Slices",
            "price": "$2.50",
            "description": "Thinly sliced zucchini can replace traditional pasta for a low-carb version.",
            "category": "Vegetables",
            "emoji": "🥒",
        },
        {
            "title": "Eggplant Slices",
            "price": "$2.75",
            "description": "Thin slices of eggplant provide a meaty texture, ideal for vegetarian lasagna.",
            "category": "Vegetables",
            "emoji": "🍆",
        },
        {
            "title": "Ground Turkey",
            "price": "$4.50",
            "description": "A leaner alternative to beef, turkey provides a lighter but flavorful taste.",
            "category": "Meat",
            "emoji": "🦃",
        },
        {
            "title": "Vegetarian Lentil Mince",
            "price": "$3.50",
            "description": "A meatless option made with cooked lentils that mimics the texture of ground meat.",
            "category": "Vegetarian",
            "emoji": "🍲",
        },
        {
            "title": "Mushroom and Walnut Mince",
            "price": "$5.00",
            "description": "Combining chopped mushrooms and walnuts for a hearty vegetarian filling.",
            "category": "Vegetarian",
            "emoji": "🍄🥜",
        },
        {
            "title": "Ground Chicken",
            "price": "$4.00",
            "description": "Ground chicken offers a different twist on the classic meat sauce.",
            "category": "Poultry",
            "emoji": "🐔",
        },
        {
            "title": "Vegan Soy Meat Crumbles",
            "price": "$4.50",
            "description": "Made from soy protein, these crumbles replicate the texture and flavor of traditional meat.",
            "category": "Vegan",
            "emoji": "🥩",
        },
        {
            "title": "Pesto Sauce",
            "price": "$4.00",
            "description": "A green, aromatic sauce made from basil, pine nuts, and garlic.",
            "category": "Canned Goods",
            "emoji": "🌿",
        },
        {
            "title": "Marinara Sauce",
            "price": "$3.50",
            "description": "A classic Italian tomato sauce with garlic, onions, and herbs.",
            "category": "Canned Goods",
            "emoji": "🍅",
        },
        {
            "title": "Bolognese Sauce",
            "price": "$5.00",
            "description": "A meat-based sauce simmered with tomatoes, onions, celery, and carrots.",
            "category": "Canned Goods",
            "emoji": "🍖🍅🧅🥕",
        },
        {
            "title": "Arrabbiata Sauce",
            "price": "$4.00",
            "description": "A spicy tomato sauce made with red chili peppers.",
            "category": "Canned Goods",
            "emoji": "🌶️🍅",
        },
        {
            "title": "Provolone Cheese",
            "price": "$3.50",
            "description": "Semi-hard cheese with a smooth texture, it melts beautifully in dishes.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Cheddar Cheese",
            "price": "$3.00",
            "description": "A popular cheese with a sharp and tangy flavor profile.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Gouda Cheese",
            "price": "$4.50",
            "description": "A Dutch cheese known for its rich and creamy texture.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Fontina Cheese",
            "price": "$4.00",
            "description": "A semi-soft cheese with a strong flavor, great for melting.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Vegan Mozzarella",
            "price": "$5.00",
            "description": "Dairy-free alternative made from nuts or soy, melts similarly to regular mozzarella.",
            "category": "Vegan",
            "emoji": "🧀",
        },
        {
            "title": "Cottage Cheese",
            "price": "$2.50",
            "description": "A lighter alternative to ricotta, with small curds that provide a similar texture.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Goat Cheese",
            "price": "$4.00",
            "description": "A tangy and creamy cheese that can provide a unique flavor to lasagna.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Mascarpone Cheese",
            "price": "$4.50",
            "description": "An Italian cream cheese with a rich and creamy texture.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Tofu Ricotta",
            "price": "$3.00",
            "description": "A vegan alternative made from crumbled tofu seasoned with herbs.",
            "category": "Vegan",
            "emoji": "🌱",
        },
        {
            "title": "Feta Cheese",
            "price": "$3.50",
            "description": "A crumbly cheese with a salty profile, it can bring a Mediterranean twist to the dish.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Parmesan cheese",
            "price": "$4.00",
            "description": "A hard, granular cheese originating from Italy, known for its rich umami flavor.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Pecorino Romano",
            "price": "$5.00",
            "description": "A salty, hard cheese made from sheep's milk, perfect for grating over dishes.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Asiago Cheese",
            "price": "$4.50",
            "description": "Semi-hard cheese with a nutty flavor, great for shaving or grating.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Grana Padano",
            "price": "$5.50",
            "description": "A grainy, hard cheese that's similar to Parmesan but milder in flavor.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Manchego Cheese",
            "price": "$6.00",
            "description": "A Spanish hard cheese with a rich and nutty flavor.",
            "category": "Dairy",
            "emoji": "🧀",
        },
        {
            "title": "Eggs",
            "price": "$2.00",
            "description": "Rich in protein and versatile, eggs are used in a variety of culinary applications.",
            "category": "Dairy",
            "emoji": "🥚",
        },
        {
            "title": "Tofu",
            "price": "$2.00",
            "description": "Blended silken tofu can act as a binder in various dishes.",
            "category": "Vegan",
            "emoji": "🍲",
        },
        {
            "title": "Flaxseed Meal",
            "price": "$1.50",
            "description": "Mix with water to create a gel-like consistency that can replace eggs.",
            "category": "Vegan",
            "emoji": "🥚",
        },
        {
            "title": "Chia Seeds",
            "price": "$2.50",
            "description": "Mix with water to form a gel that can be used as an egg substitute.",
            "category": "Vegan",
            "emoji": "🥚",
        },
        {
            "title": "Apple Sauce",
            "price": "$2.00",
            "description": "A sweet alternative that can replace eggs in certain recipes.",
            "category": "Baking",
            "emoji": "🥚",
        },
        {
            "title": "Onion",
            "price": "$1.00",
            "description": "A kitchen staple, onions provide depth and flavor to a myriad of dishes.",
            "category": "Vegetables",
            "emoji": "🧅",
        },
        {
            "title": "Shallots",
            "price": "$2.00",
            "description": "Milder and sweeter than regular onions, they add a delicate flavor.",
            "category": "Produce",
            "emoji": "🧅",
        },
        {
            "title": "Green Onions",
            "price": "$1.50",
            "description": "Milder in flavor, green onions or scallions are great for garnishing.",
            "category": "Vegetables",
            "emoji": "🌱",
        },
        {
            "title": "Red Onion",
            "price": "$1.20",
            "description": "Sweeter and more vibrant in color, red onions add a pop to dishes.",
            "category": "Vegetables",
            "emoji": "🔴",
        },
        {
            "title": "Leeks",
            "price": "$2.50",
            "description": "With a light onion flavor, leeks are great when sautéed or used in soups.",
            "category": "Produce",
            "emoji": "🍲",
        },
        {
            "title": "Garlic",
            "price": "$0.50",
            "description": "Aromatic and flavorful, garlic is a foundational ingredient in many cuisines.",
            "category": "Produce",
            "emoji": "🧄",
        },
        {
            "title": "Garlic Powder",
            "price": "$2.00",
            "description": "A convenient dried version of garlic that provides a milder flavor.",
            "category": "Spices",
            "emoji": "🧄",
        },
        {
            "title": "Garlic Flakes",
            "price": "$2.50",
            "description": "Dried garlic flakes can be rehydrated or used as they are for a burst of garlic flavor.",
            "category": "Spices",
            "emoji": "🧄",
        },
        {
            "title": "Garlic Paste",
            "price": "$3.00",
            "description": "A smooth blend of garlic, perfect for adding to sauces or marinades.",
            "category": "Condiments",
            "emoji": "🧄",
        },
        {
            "title": "Olive Oil",
            "price": "$6.00",
            "description": "A staple in Mediterranean cuisine, olive oil is known for its heart-healthy properties.",
            "category": "Condiments",
            "emoji": "🍽️",
        },
        {
            "title": "Canola Oil",
            "price": "$3.50",
            "description": "A neutral-tasting oil suitable for various cooking methods.",
            "category": "Condiments",
            "emoji": "🍳",
        },
        {
            "title": "Coconut Oil",
            "price": "$5.00",
            "description": "A fragrant oil ideal for sautéing and baking.",
            "category": "Condiments",
            "emoji": "🍳",
        },
        {
            "title": "Avocado Oil",
            "price": "$7.00",
            "description": "Known for its high smoke point, it's great for high-heat cooking.",
            "category": "Condiments",
            "emoji": "🍳",
        },
        {
            "title": "Grapeseed Oil",
            "price": "$6.50",
            "description": "A light, neutral oil that's good for dressings and sautéing.",
            "category": "Condiments",
            "emoji": "🥗",
        },
        {
            "title": "Salt",
            "price": "$1.00",
            "description": "An essential seasoning that enhances the flavor of dishes.",
            "category": "Spices",
            "emoji": "🧂",
        },
        {
            "title": "Himalayan Pink Salt",
            "price": "$2.50",
            "description": "A natural and unrefined salt with a slightly earthy flavor.",
            "category": "Spices",
            "emoji": "🧂",
        },
        {
            "title": "Sea Salt",
            "price": "$2.00",
            "description": "Derived from evaporated seawater, it provides a briny touch.",
            "category": "Spices",
            "emoji": "🌊",
        },
        {
            "title": "Kosher Salt",
            "price": "$1.50",
            "description": "A coarse salt without additives, commonly used in cooking.",
            "category": "Spices",
            "emoji": "🧂",
        },
        {
            "title": "Black Salt (Kala Namak)",
            "price": "$2.00",
            "description": "A sulfurous salt often used in South Asian cuisine, especially vegan dishes to mimic an eggy flavor.",
            "category": "Spices",
            "emoji": "🧂",
        },
        {
            "title": "Black Pepper",
            "price": "$2.00",
            "description": "A versatile spice known for its sharp and mildly spicy flavor.",
            "category": "Spices",
            "emoji": "🌶️",
        },
        {
            "title": "White Pepper",
            "price": "$2.50",
            "description": "Milder than black pepper, it's often used in light-colored dishes.",
            "category": "Spices",
            "emoji": "🌶️",
        },
        {
            "title": "Cayenne Pepper",
            "price": "$2.00",
            "description": "A spicy chili pepper, ground into powder. Adds heat to dishes.",
            "category": "Spices",
            "emoji": "🌶️",
        },
        {
            "title": "Crushed Red Pepper Flakes",
            "price": "$1.50",
            "description": "Adds a spicy kick to dishes, commonly used as a pizza topping.",
            "category": "Spices",
            "emoji": "🌶️",
        },
        {
            "title": "Sichuan (or Szechuan) Peppercorns",
            "price": "$3.00",
            "description": "Known for their unique tingling sensation, they're used in Chinese cuisine.",
            "category": "Spices",
            "emoji": "🥡",
        },
        {
            "title": "Banana",
            "price": "$0.60",
            "description": "A sweet and portable fruit, packed with essential vitamins.",
            "category": "Produce",
            "emoji": "🍌",
        },
        {
            "title": "Milk",
            "price": "$2.50",
            "description": "A calcium-rich dairy product, perfect for drinking or cooking.",
            "category": "Dairy",
            "emoji": "🥛",
        },
        {
            "title": "Bread",
            "price": "$2.00",
            "description": "Freshly baked, perfect for sandwiches or toast.",
            "category": "Bakery",
            "emoji": "🍞",
        },
        {
            "title": "Apple",
            "price": "$1.00",
            "description": "Crisp and juicy, great for snacking or baking.",
            "category": "Produce",
            "emoji": "🍏",
        },
        {
            "title": "Orange",
            "price": "3.99$",
            "description": "Great as a juice and vitamin",
            "category": "Produce",
            "emoji": "🍊",
        },
        {
            "title": "Sugar",
            "price": "1.00",
            "description": "very sweet substance",
            "category": "Spices",
            "emoji": "🍰",
        },
    ]
}

insert_data = []

for product in dataset["train"]:
    doc_product = json_util.loads(json_util.dumps(product))
    haystack_doc = Document(content=doc_product["title"], meta=doc_product)
    insert_data.append(haystack_doc)


document_store = MongoDBAtlasDocumentStore(
    database_name="ai_shop",
    collection_name="test_collection",
    vector_search_index="vector_index",
)

Build the writer pipeline to load documnets

In [ ]:
# Setting up a document writer to handle the insertion of documents into the MongoDB collection.
doc_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

# Initializing a document embedder to convert text content into vectorized form.
doc_embedder = OpenAIDocumentEmbedder()

# Creating a pipeline for indexing documents. The pipeline includes embedding and writing documents.
indexing_pipe = Pipeline()
indexing_pipe.add_component(instance=doc_embedder, name="doc_embedder")
indexing_pipe.add_component(instance=doc_writer, name="doc_writer")

# Connecting the components of the pipeline for document flow.
indexing_pipe.connect("doc_embedder.documents", "doc_writer.documents")

# Running the pipeline with the list of documents to index them in MongoDB.
indexing_pipe.run({"doc_embedder": {"documents": insert_data}})

Calculating embeddings: 100%|██████████| 3/3 [00:01<00:00,  2.85it/s]


{'doc_embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 310, 'total_tokens': 310}}},
 'doc_writer': {'documents_written': 81}}

## Build a RAG Pipeline

Lets create a pipeline that will Retrieve Augment and Generate a response for user questions

In [ ]:
# Template for generating prompts for a movie recommendation engine.
prompt_template = """
    You are a chef assistant allowed to use the following context documents and only those.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \Query: {{query}}

    \nAnswer:
"""

# Setting up a retrieval-augmented generation (RAG) pipeline for generating responses.
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", OpenAITextEmbedder())

# Adding a component for retrieving related documents from MongoDB based on the query embedding.
rag_pipeline.add_component(
    instance=MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=50),
    name="retriever",
)

# Building prompts based on retrieved documents to be used for generating responses.
rag_pipeline.add_component(
    instance=PromptBuilder(template=prompt_template), name="prompt_builder"
)

# Adding a language model generator to produce the final text output.
rag_pipeline.add_component(instance=OpenAIGenerator(model="gpt-4o"), name="llm")

# Connecting the components of the RAG pipeline to ensure proper data flow.
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: MongoDBAtlasEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

Lets test the pipeline

In [ ]:
query = "How can I cook a lasagne?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": query},
        "prompt_builder": {"query": query},
    }
)
print(result["llm"]["replies"][0])

To cook a lasagne, you can follow this classic recipe:

### Ingredients:
#### For the meat sauce:
- 2 tablespoons olive oil
- 1 onion, finely chopped
- 2 cloves garlic, minced
- 500g ground beef
- 800g canned tomatoes, crushed
- 2 tablespoons tomato paste
- 1 teaspoon dried basil
- 1 teaspoon dried oregano
- Salt and pepper to taste

#### For the béchamel sauce:
- 4 tablespoons butter
- 4 tablespoons all-purpose flour
- 500ml milk
- A pinch of nutmeg
- Salt and pepper to taste

#### For assembly:
- 250g lasagne sheets
- 200g mozzarella cheese, shredded
- 1 cup grated Parmesan cheese
- Fresh basil leaves for garnish (optional)

### Instructions:
1. **Preheat the oven** to 375°F (190°C).

2. **Prepare the meat sauce:**
   - Heat the olive oil in a large skillet over medium heat.
   - Add the chopped onion and cook until soft and translucent, about 5 minutes.
   - Stir in the minced garlic and cook for another minute.
   - Add the ground beef and cook until browned, breaking it up with a 